In [2]:
import keras
import numpy as np
import matplotlib as plt 

from keras.layers import Dense, Dropout, BatchNormalization, Convolution1D, Flatten, Lambda, Permute, MaxPooling1D, AlphaDropout
from keras.models import Sequential
from keras.utils import to_categorical
import keras.backend as K

Using TensorFlow backend.


In [3]:
(a,b) = keras.datasets.mnist.load_data()

In [4]:
x_train = a[0]
y_train = to_categorical(a[1])

x_test = b[0]
y_test = to_categorical(b[1])

In [5]:
x_train.shape

(60000, 28, 28)

In [165]:
m1 = Sequential([
    Permute((2,1), input_shape=(28,28)),
    BatchNormalization(),
    Convolution1D(32, (3), activation='relu', padding='same'),
    Dropout(0.1),
    MaxPooling1D(2),
    BatchNormalization(),
    Convolution1D(64, (3), activation='relu', padding='same'),
    Dropout(0.1),
    MaxPooling1D(2),
    Flatten(),
    BatchNormalization(),
    Dense(20, activation='relu'),
    BatchNormalization(),
    Dense(10, activation='softmax')   
])

m1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [166]:
m1.fit(x_train, y_train, epochs=10)

Epoch 1/1
60000/60000 [==============================] - 36s 598us/step - loss: 0.0527 - acc: 0.9823


In [ ]:
m1.optimizer.lr = 0.00001

In [364]:
m1.fit(x_train, y_train, epochs=10)

Epoch 1/10
60000/60000 [==============================] - 33s 558us/step - loss: 0.0073 - acc: 0.9975
Epoch 2/10
60000/60000 [==============================] - 32s 536us/step - loss: 0.0071 - acc: 0.9975
Epoch 3/10
60000/60000 [==============================] - 37s 611us/step - loss: 0.0072 - acc: 0.9974
Epoch 4/10
60000/60000 [==============================] - 34s 565us/step - loss: 0.0069 - acc: 0.9975
Epoch 5/10
60000/60000 [==============================] - 29s 491us/step - loss: 0.0066 - acc: 0.9976
Epoch 6/10
60000/60000 [==============================] - 31s 510us/step - loss: 0.0064 - acc: 0.9978
Epoch 7/10
60000/60000 [==============================] - 27s 446us/step - loss: 0.0066 - acc: 0.9976
Epoch 8/10
60000/60000 [==============================] - 31s 516us/step - loss: 0.0066 - acc: 0.9977
Epoch 9/10
60000/60000 [==============================] - 28s 463us/step - loss: 0.0063 - acc: 0.9977
Epoch 10/10
60000/60000 [==============================] - 32s 532us/step - loss: 

In [365]:
m1.evaluate(x_test, y_test)


10000/10000 [==============================] - 2s 162us/step


[0.0078690886313561356, 0.99738999557495123]

In [179]:
# from keras.models import load_model

# m1.save_weights('m1_mnist_ensemble2.h5')

In [180]:
m2 = Sequential([
    BatchNormalization(input_shape=(28,28)),
    Convolution1D(32, (3), activation='relu', padding='same'),
    Dropout(0.1),
    MaxPooling1D(2),
    BatchNormalization(),
    Convolution1D(64, (3), activation='relu', padding='same'),
    Dropout(0.1),
    MaxPooling1D(2),
    Flatten(),
    BatchNormalization(),
    Dense(20, activation='relu'),
    BatchNormalization(),
    Dense(10, activation='softmax')   
])

m2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [181]:
m2.fit(x_train, y_train, epochs=10)
m2.optimizer.lr = 0.00001
m2.fit(x_train, y_train, epochs=10)
m2.evaluate(x_test, y_test)

Epoch 1/10
60000/60000 [==============================] - 33s 557us/step - loss: 0.0485 - acc: 0.9838
Epoch 2/10
60000/60000 [==============================] - 36s 595us/step - loss: 0.0184 - acc: 0.9938
Epoch 3/10
60000/60000 [==============================] - 32s 529us/step - loss: 0.0146 - acc: 0.9950
Epoch 4/10
60000/60000 [==============================] - 36s 596us/step - loss: 0.0123 - acc: 0.9958
Epoch 5/10
60000/60000 [==============================] - 31s 521us/step - loss: 0.0114 - acc: 0.9961
Epoch 6/10
60000/60000 [==============================] - 34s 569us/step - loss: 0.0103 - acc: 0.9965
Epoch 7/10
60000/60000 [==============================] - 35s 588us/step - loss: 0.0095 - acc: 0.9967
Epoch 8/10
60000/60000 [==============================] - 36s 599us/step - loss: 0.0091 - acc: 0.9969
Epoch 9/10
60000/60000 [==============================] - 35s 582us/step - loss: 0.0086 - acc: 0.9972
Epoch 10/10
60000/60000 [==============================] - 35s 579us/step - loss: 

[0.0067760217707051196, 0.99789999694824216]

## Create ensemble data

In [245]:
# Training
m1_pred = m1.predict(x_train)
m2_pred = m2.predict(x_train)
ens_data = np.array([m1_pred, m2_pred])
ens_data = np.swapaxes(ens_data, 0, 1)
ens_data = np.swapaxes(ens_data, 1, 2)

# Testing
test_m1_pred = m1.predict(x_test)
test_m2_pred = m2.predict(x_test)
test_ens_data = np.array([test_m1_pred, test_m2_pred])
test_ens_data = np.swapaxes(test_ens_data, 0, 1)
test_ens_data = np.swapaxes(test_ens_data, 1, 2)

In [323]:
print(test_ens_data.shape)
print(ens_data.shape)
ens_data = np.round(ens_data)
test_ens_data = np.round(test_ens_data)

(10000, 10, 2)
(60000, 10, 2)


In [355]:
print(m1.evaluate(x_test, y_test))
print(m2.evaluate(x_test, y_test))

10000/10000 [==============================] - 2s 216us/step
[0.0073353876278793905, 0.99753999576568608]
10000/10000 [==============================] - 2s 209us/step
[0.0067760217707051196, 0.99789999694824216]


## Create ensemble model

In [362]:
e1 = Sequential([
    Flatten(input_shape=(10,2)),
    AlphaDropout(0.8),
    Dense(10, activation='softmax'),
])

e1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


## Train ensemble model

In [363]:
e1.fit(ens_data, y_train,epochs=4, validation_data=(test_ens_data, y_test))
e1.optimizer.lr = 0.00001
e1.fit(ens_data, y_train,epochs=10, validation_data=(test_ens_data, y_test))


Train on 60000 samples, validate on 10000 samples
Epoch 1/4
60000/60000 [==============================] - 12s 204us/step - loss: 2.3394 - acc: 0.1783 - val_loss: 1.8929 - val_acc: 0.9891
Epoch 2/4
60000/60000 [==============================] - 7s 109us/step - loss: 2.2477 - acc: 0.2598 - val_loss: 1.8643 - val_acc: 0.9895
Epoch 3/4
60000/60000 [==============================] - 6s 104us/step - loss: 2.2495 - acc: 0.2551 - val_loss: 1.8803 - val_acc: 0.9879
Epoch 4/4
36416/60000 [=================>............] - ETA: 3s - loss: 2.2490 - acc: 0.2555

KeyboardInterrupt: 

In [351]:
e1.evaluate(test_ens_data, y_test)

10000/10000 [==============================] - 1s 61us/step


[0.12752548183202744, 0.98909999999999998]

10000/10000 [==============================] - 2s 218us/step


[0.0073353876278793905, 0.99753999576568608]

10000/10000 [==============================] - 2s 196us/step


[0.0067760217707051196, 0.99789999694824216]